<a href="https://colab.research.google.com/github/daniel-hain/paper_skill_development_NO/blob/main/10_embeddings_sbert_jobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies needed for this notebook

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!pip install -qU transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.3 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, util

In [13]:
import pickle
import pandas as pd
import numpy as np

import pyarrow as pa
import pyarrow.feather as feather
import pyarrow.parquet as pq

In [5]:
embeddings_dir = "/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/embeddings_folder"

---

## Using sentence-transformers

In [6]:
model = SentenceTransformer('NbAiLab/nb-sbert-base')

# Embedding jobs

In [7]:
# Read the CSV file
df_jobs  = pd.read_csv("/content/gdrive/MyDrive/00_projects/project_2023_skills_NO/text_job_plain.csv")

In [8]:
from more_itertools import sliced
CHUNK_SIZE = 10000  #chunk row size
index_slices = sliced(range(len(df_jobs)), CHUNK_SIZE)

In [ ]:
for index_slice in index_slices:
  chunk = df_jobs.iloc[index_slice]

  embeddings = model.encode(chunk['text'].tolist(), show_progress_bar=True, convert_to_tensor=False, convert_to_numpy=True)

  df_embeddings = pd.DataFrame(embeddings)
  df_embeddings = df_embeddings.set_index(chunk['job_id'])
  df_embeddings = pa.Table.from_pandas(df_embeddings)
  pq.write_table(df_embeddings, f"{embeddings_dir}/job_embeddings_{index_slice}.parquet")

  print("::::: Finished chunk ", index_slice, ':::::')

In [ ]:
#df_embeddings = pd.DataFrame(embeddings)
#df_embeddings = df_embeddings.set_index(chunk['job_id'])
#df_embeddings = pa.Table.from_pandas(df_embeddings)
#pq.write_table(df_embeddings, f"{embeddings_dir}/job_embeddings_{index_slice}.parquet")